In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_binance_futures = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binance_futures.csv')
df_binance_futures = df_binance_futures.assign(exchange_id = 'binance_futures')
df_binance_futures.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.406,5.406,5.401,5.403,20601.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.403,5.404,5.401,5.403,7417.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.403,5.407,5.403,5.407,4555.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.406,5.408,5.404,5.405,11194.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.404,5.406,5.404,5.405,1942.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10


In [4]:
df_binance_spot = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binance_spot.csv')
df_binance_spot = df_binance_spot.assign(exchange_id = 'binance_spot')
df_binance_spot.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.408,5.408,5.404,5.404,726.43,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.404,5.406,5.403,5.406,838.73,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.406,5.409,5.406,5.407,745.08,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.407,5.409,5.405,5.406,2627.81,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.405,5.407,5.405,5.407,201.09,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10


In [5]:
df_binance_us = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binanceus.csv')
df_binance_us.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10


In [6]:
df_okx = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_okx.csv')
df_okx.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 15:00:00+00:00,1664636400000,5.3401,5.3401,5.3361,5.3361,189.878030,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
1,2022-10-01 15:01:00+00:00,1664636460000,5.3356,5.3356,5.3262,5.3262,1404.094091,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2,2022-10-01 15:02:00+00:00,1664636520000,5.3273,5.3318,5.3273,5.3313,1525.072300,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
3,2022-10-01 15:03:00+00:00,1664636580000,5.3316,5.3379,5.3316,5.3354,530.874701,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
4,2022-10-01 15:04:00+00:00,1664636640000,5.3348,5.3348,5.3303,5.3303,259.569769,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10


let's work with a small sample size of data: october 2022 in okx exchange, and once we figure out how to do stuff, then we can scale upwards

In [6]:
temp = df_okx.loc[((df_okx['year'] == 2022) & (df_okx['month'] == 10))]
temp.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 15:00:00+00:00,1664636400000,5.3401,5.3401,5.3361,5.3361,189.878030,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
1,2022-10-01 15:01:00+00:00,1664636460000,5.3356,5.3356,5.3262,5.3262,1404.094091,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2,2022-10-01 15:02:00+00:00,1664636520000,5.3273,5.3318,5.3273,5.3313,1525.072300,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
3,2022-10-01 15:03:00+00:00,1664636580000,5.3316,5.3379,5.3316,5.3354,530.874701,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
4,2022-10-01 15:04:00+00:00,1664636640000,5.3348,5.3348,5.3303,5.3303,259.569769,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10


In [7]:
temp.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'], inplace=True)

In [8]:
grouped = temp.groupby('currency_pair')
dfs = [grouped.get_group(x) for x in grouped.groups]

In [9]:
merged_okx = dfs[0]['timestamp'] # save this for later, will help with merging
timestamp = dfs[0]['timestamp']

In [10]:
for df in dfs:
    df['vwap-okx::'+df['currency_pair'].unique()[0]] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
    df.rename(columns={"volume" : 'volume-okx::'+df['currency_pair'].unique()[0]}, inplace=True)
    df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
    df.set_index('timestamp', inplace=True)

it looks like the above was done correctly -> now we have to try and merge all of them

In [11]:
for df in dfs:
    merged_okx = pd.merge(merged_okx, df, left_on='timestamp', right_on='timestamp')

In [12]:
merged_okx = merged_okx.set_index('timestamp')
merged_okx

,volume-okx::APE_USDT,vwap-okx::APE_USDT,volume-okx::AVAX_USDT,vwap-okx::AVAX_USDT,volume-okx::AXS_USDT,vwap-okx::AXS_USDT,volume-okx::BTC_USDT,vwap-okx::BTC_USDT,volume-okx::DOGE_USDT,vwap-okx::DOGE_USDT,...,volume-okx::LINK_USDT,vwap-okx::LINK_USDT,volume-okx::NEAR_USDT,vwap-okx::NEAR_USDT,volume-okx::SAND_USDT,vwap-okx::SAND_USDT,volume-okx::SOL_USDT,vwap-okx::SOL_USDT,volume-okx::XRP_USDT,vwap-okx::XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-10-01 15:00:00+00:00,189.878030,5.338100,28.726003,17.185000,17.767561,12.640000,0.249802,19321.050000,1.558154e+04,0.061460,...,734.434521,7.520000,2079.172278,3.555500,1049.994113,0.837650,1.371908,32.890000,8668.530595,0.476615
2022-10-01 15:01:00+00:00,1404.094091,5.331758,64.657472,17.178076,2.010417,12.640000,7.931582,19315.136251,1.224445e+05,0.061398,...,546.093082,7.517868,5514.181050,3.553685,4191.800000,0.837330,673.490763,32.855071,45756.333858,0.476493
2022-10-01 15:02:00+00:00,1525.072300,5.330678,0.011905,17.178074,0.000000,12.640000,0.372186,19314.864897,3.682257e+04,0.061389,...,6.357715,7.517846,59.287817,3.553656,520.495735,0.837264,208.113192,32.853876,36390.661517,0.476490
2022-10-01 15:03:00+00:00,530.874701,5.331270,26.644861,17.176282,0.178015,12.640089,2.130541,19315.440221,2.759960e+04,0.061391,...,201.010928,7.518137,983.510440,3.553524,1930.253027,0.837248,38.984093,32.854558,32015.369617,0.476651
2022-10-01 15:04:00+00:00,259.569769,5.331355,40.827680,17.173418,0.000000,12.640089,1.153486,19315.787095,6.177918e+04,0.061386,...,82.386709,7.518209,2128.638720,3.552827,4378.745562,0.836977,42.055153,32.855231,111647.561655,0.477084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 13:15:00+00:00,2220.890405,4.753914,31.467091,16.981017,161.662144,9.672030,0.297364,19681.670574,1.332704e+06,0.093484,...,681.567466,7.223080,1689.795082,3.097338,24316.110275,0.795820,76.328451,31.080670,51207.442613,0.476448
2022-10-31 13:16:00+00:00,490.571188,4.753923,161.070076,16.981798,54.342640,9.671984,0.184068,19681.678820,7.392846e+05,0.093500,...,751.366484,7.223396,1659.744407,3.097357,4352.069129,0.795835,30.438649,31.080780,54935.098462,0.476444
2022-10-31 13:17:00+00:00,598.120377,4.753934,489.361336,16.984164,236.596097,9.671783,0.619490,19681.706612,9.256884e+05,0.093520,...,508.607888,7.223611,475.013401,3.097363,8312.630257,0.795863,165.018809,31.081377,62034.089256,0.476440


In [7]:
# create helper function
def create_single(exchange_df):
    exchange_df = exchange_df.sort_values(by='timestamp')
    id = exchange_df['exchange_id'].unique()[0]
    exchange_df = exchange_df.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'])
    grouped = exchange_df.groupby('currency_pair')
    dfs = [grouped.get_group(x) for x in grouped.groups]
    merged = dfs[0]['timestamp'] # save this for later, will help with merging
    for df in dfs:
        df[f"vwap-{id}:{df['currency_pair'].unique()[0]}"] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
        df.rename(columns={'volume' : 'volume-' + id +':'+df['currency_pair'].unique()[0]}, inplace=True)
        df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
        df.set_index('timestamp', inplace=True)
    for df in dfs:
        merged = pd.merge(merged, df, left_on='timestamp', right_on='timestamp')
    timestamp = merged['timestamp']
    merged = merged.set_index('timestamp')
    return merged.sort_index(axis=1), timestamp

In [8]:
single, t = create_single(df_okx)
single

,volume-okx:APE_USDT,volume-okx:AVAX_USDT,volume-okx:AXS_USDT,volume-okx:BNB_USDT,volume-okx:BTC_USDT,volume-okx:DOGE_USDT,volume-okx:DOT_USDT,volume-okx:ETH_USDT,volume-okx:FTM_USDT,volume-okx:GMT_USDT,...,vwap-okx:FTM_USDT,vwap-okx:GMT_USDT,vwap-okx:LINK_USDT,vwap-okx:MATIC_USDT,vwap-okx:NEAR_USDT,vwap-okx:SAND_USDT,vwap-okx:SOL_USDT,vwap-okx:STORJ_USDT,vwap-okx:WAVES_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-01 15:00:00+00:00,143.559420,10.223912,5.745520,1.409575,0.325563,2.384914e+05,355.671048,167.288462,7516.504437,249.6042,...,1.280214,1.083322,21.986399,1.287627,7.396922,1.519536,58.248381,1.358559,15.465800,0.680726
2023-01-01 15:01:00+00:00,0.000000,0.958191,0.000000,3.170915,0.032026,7.210098e+03,0.000000,8.334945,0.000000,0.0000,...,1.280214,1.083322,21.986397,1.287627,7.396922,1.519535,58.248369,1.358559,15.465800,0.680725
2023-01-01 15:02:00+00:00,0.000000,0.010050,0.000000,3.585290,0.279657,2.218515e+03,0.000000,2.583366,0.000000,0.0000,...,1.280214,1.083322,21.986396,1.287627,7.396910,1.519535,58.248303,1.358555,15.465800,0.680724
2023-01-01 15:03:00+00:00,68.454178,0.010238,0.000000,7.773471,0.252629,1.042193e+03,0.000000,0.971597,0.000000,0.0000,...,1.280214,1.083322,21.986396,1.287627,7.396910,1.519535,58.248300,1.358555,15.465800,0.680724
2023-01-01 15:04:00+00:00,319.673310,20.230825,0.000000,3.543093,0.088499,1.853383e+04,6.381349,3.106249,0.000000,528.5635,...,1.280214,1.083320,21.986393,1.287627,7.396910,1.519534,58.248206,1.358554,15.465772,0.680717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 13:15:00+00:00,748.782823,13.863603,0.000000,6.830231,0.657629,1.499997e+05,3222.292738,66.025269,15002.460617,2312.0000,...,1.246672,1.029788,21.705751,1.283227,7.209707,1.480779,56.795526,1.338541,14.483248,0.667274
2023-01-31 13:16:00+00:00,1711.776069,16.577327,45.360064,1.091403,0.857525,1.022419e+06,42.302094,318.450571,5684.622391,1820.0000,...,1.246670,1.029785,21.705744,1.283227,7.209700,1.480774,56.795422,1.338539,14.483180,0.667273
2023-01-31 13:17:00+00:00,512.456285,1.870275,4.759460,2.238450,3.007150,4.666947e+05,295.076435,137.433309,38902.862178,2221.3500,...,1.246657,1.029781,21.705736,1.283227,7.209685,1.480773,56.795367,1.338539,14.482937,0.667272


# can we get this as a multi-index df
in fact, we will use this normal df and convert it into multiindex

In [11]:
len(list(single.columns))

36

In [28]:
cols_str = ""
cols = list(single.columns)
for elem in cols:
    cols_str = "".join([cols_str, elem]) + " "
len(cols_str.split())/2


18.0

In [42]:
features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * 18
exchange_str = ""
for col in cols:
    h, s = col.rfind('-') + 1, col.rfind(':')
    exchange_str += col[h:s] + " "
timestamp = single.index

In [47]:
len(exchange_str.split())

36

In [14]:
multi = pd.DataFrame(np.array(single), columns=[features_str.split(), exchange_str.split(), cols_str.split()])
multi.set_index(timestamp, inplace=True)

In [16]:
multi.head()

volume                       \
                                          okx                        
                          volume-okx:APE_USDT volume-okx:AVAX_USDT   
timestamp                                                            
2023-01-01 15:00:00+00:00          143.559420            10.223912   
2023-01-01 15:01:00+00:00            0.000000             0.958191   
2023-01-01 15:02:00+00:00            0.000000             0.010050   
2023-01-01 15:03:00+00:00           68.454178             0.010238   
2023-01-01 15:04:00+00:00          319.673310            20.230825   

                                                                   \
                                                                    
                          volume-okx:AXS_USDT volume-okx:BNB_USDT   
timestamp                                                           
2023-01-01 15:00:00+00:00             5.74552            1.409575   
2023-01-01 15:01:00+00:00             0.00000            3.170915   
2023-01-01 15:02:00+00:00             0.00000            3.585290   
2023-01-01 15:03:00+00:00             0.00000            7.773471   
2023-01-01 15:04:00+00:00             0.00000            3.543093   

                                                                    \
                                                                     
                          volume-okx:BTC_USDT volume-okx:DOGE_USDT   
timestamp                                                            
2023-01-01 15:00:00+00:00            0.325563        238491.424176   
2023-01-01 15:01:00+00:00            0.032026          7210.098280   
2023-01-01 15:02:00+00:00            0.279657          2218.514779   
2023-01-01 15:03:00+00:00            0.252629          1042.192843   
2023-01-01 15:04:00+00:00            0.088499         18533.832141   

                                                                   \
                                                                    
                          volume-okx:DOT_USDT volume-okx:ETH_USDT   
timestamp                                                           
2023-01-01 15:00:00+00:00          355.671048          167.288462   
2023-01-01 15:01:00+00:00            0.000000            8.334945   
2023-01-01 15:02:00+00:00            0.000000            2.583366   
2023-01-01 15:03:00+00:00            0.000000            0.971597   
2023-01-01 15:04:00+00:00            6.381349            3.106249   

                                                                   ...  \
                                                                   ...   
                          volume-okx:FTM_USDT volume-okx:GMT_USDT  ...   
timestamp                                                          ...   
2023-01-01 15:00:00+00:00         7516.504437            249.6042  ...   
2023-01-01 15:01:00+00:00            0.000000              0.0000  ...   
2023-01-01 15:02:00+00:00            0.000000              0.0000  ...   
2023-01-01 15:03:00+00:00            0.000000              0.0000  ...   
2023-01-01 15:04:00+00:00            0.000000            528.5635  ...   

                                       vwap                    \
                                        okx                     
                          vwap-okx:FTM_USDT vwap-okx:GMT_USDT   
timestamp                                                       
2023-01-01 15:00:00+00:00          1.280214          1.083322   
2023-01-01 15:01:00+00:00          1.280214          1.083322   
2023-01-01 15:02:00+00:00          1.280214          1.083322   
2023-01-01 15:03:00+00:00          1.280214          1.083322   
2023-01-01 15:04:00+00:00          1.280214          1.083320   

                                                                  \
                                                                   
                          vwap-okx:LINK_USDT vwap-okx:MATIC_USDT   
timestamp                                                          
2023-01-

In [18]:
d = dict(zip(multi.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in multi.columns]))
multi = multi.rename(columns=d, level=2)
multi.head()

volume                                          \
                                  okx                                           
                             APE_USDT  AVAX_USDT AXS_USDT  BNB_USDT  BTC_USDT   
timestamp                                                                       
2023-01-01 15:00:00+00:00  143.559420  10.223912  5.74552  1.409575  0.325563   
2023-01-01 15:01:00+00:00    0.000000   0.958191  0.00000  3.170915  0.032026   
2023-01-01 15:02:00+00:00    0.000000   0.010050  0.00000  3.585290  0.279657   
2023-01-01 15:03:00+00:00   68.454178   0.010238  0.00000  7.773471  0.252629   
2023-01-01 15:04:00+00:00  319.673310  20.230825  0.00000  3.543093  0.088499   

                                                                               \
                                                                                
                               DOGE_USDT    DOT_USDT    ETH_USDT     FTM_USDT   
timestamp                                                                       
2023-01-01 15:00:00+00:00  238491.424176  355.671048  167.288462  7516.504437   
2023-01-01 15:01:00+00:00    7210.098280    0.000000    8.334945     0.000000   
2023-01-01 15:02:00+00:00    2218.514779    0.000000    2.583366     0.000000   
2023-01-01 15:03:00+00:00    1042.192843    0.000000    0.971597     0.000000   
2023-01-01 15:04:00+00:00   18533.832141    6.381349    3.106249     0.000000   

                                     ...      vwap                       \
                                     ...       okx                        
                           GMT_USDT  ...  FTM_USDT  GMT_USDT  LINK_USDT   
timestamp                            ...                                  
2023-01-01 15:00:00+00:00  249.6042  ...  1.280214  1.083322  21.986399   
2023-01-01 15:01:00+00:00    0.0000  ...  1.280214  1.083322  21.986397   
2023-01-01 15:02:00+00:00    0.0000  ...  1.280214  1.083322  21.986396   
2023-01-01 15:03:00+00:00    0.0000  ...  1.280214  1.083322  21.986396   
2023-01-01 15:04:00+00:00  528.5635  ...  1.280214  1.083320  21.986393   

                                                                     \
                                                                      
                          MATIC_USDT NEAR_USDT SAND_USDT   SOL_USDT   
timestamp                                                             
2023-01-01 15:00:00+00:00   1.287627  7.396922  1.519536  58.248381   
2023-01-01 15:01:00+00:00   1.287627  7.396922  1.519535  58.248369   
2023-01-01 15:02:00+00:00   1.287627  7.396910  1.519535  58.248303   
2023-01-01 15:03:00+00:00   1.287627  7.396910  1.519535  58.248300   
2023-01-01 15:04:00+00:00   1.287627  7.396910  1.519534  58.248206   

                                                           
                                                           
                          STORJ_USDT WAVES_USDT  XRP_USDT  
timestamp                                                  
2023-01-01 15:00:00+00:00   1.358559  15.465800  0.680726  
2023-01-01 15:01:00+00:00   1.358559  15.465800  0.680725  
2023-01-01 15:02:00+00:00   1.358555  15.465800  0.680724  
2023-01-01 15:03:00+00:00   1.358555  15.465800  0.680724  
2023-01-01 15:04:00+00:00   1.358554  15.465772  0.680717  

[5 rows x 36 columns]

### the above is the correct df for all coins that exist on the okx exchange in october 2022

In [56]:
def convert_to_multi(df, timestamp):
    # creating most inner column 
    cols_str = ""
    cols = list(single.columns)
    for elem in cols:
        cols_str = "".join([cols_str, elem]) + " "
    len(cols_str.split())

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchanges_str = ""
    for col in cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchanges_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * int(len(cols_str.split())/2)

    new_df = pd.DataFrame(np.array(df), columns=[features_str.split(), exchanges_str.split(), cols_str.split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp)
    return new_df

In [57]:
single_okx, t_okx = create_single(df_okx)
single_okx.head()

,volume-okx:APE_USDT,volume-okx:AVAX_USDT,volume-okx:AXS_USDT,volume-okx:BNB_USDT,volume-okx:BTC_USDT,volume-okx:DOGE_USDT,volume-okx:DOT_USDT,volume-okx:ETH_USDT,volume-okx:FTM_USDT,volume-okx:GMT_USDT,...,vwap-okx:FTM_USDT,vwap-okx:GMT_USDT,vwap-okx:LINK_USDT,vwap-okx:MATIC_USDT,vwap-okx:NEAR_USDT,vwap-okx:SAND_USDT,vwap-okx:SOL_USDT,vwap-okx:STORJ_USDT,vwap-okx:WAVES_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-01 15:00:00+00:00,143.559420,10.223912,5.74552,1.409575,0.325563,238491.424176,355.671048,167.288462,7516.504437,249.6042,...,1.280214,1.083322,21.986399,1.287627,7.396922,1.519536,58.248381,1.358559,15.465800,0.680726
2023-01-01 15:01:00+00:00,0.000000,0.958191,0.00000,3.170915,0.032026,7210.098280,0.000000,8.334945,0.000000,0.0000,...,1.280214,1.083322,21.986397,1.287627,7.396922,1.519535,58.248369,1.358559,15.465800,0.680725
2023-01-01 15:02:00+00:00,0.000000,0.010050,0.00000,3.585290,0.279657,2218.514779,0.000000,2.583366,0.000000,0.0000,...,1.280214,1.083322,21.986396,1.287627,7.396910,1.519535,58.248303,1.358555,15.465800,0.680724
2023-01-01 15:03:00+00:00,68.454178,0.010238,0.00000,7.773471,0.252629,1042.192843,0.000000,0.971597,0.000000,0.0000,...,1.280214,1.083322,21.986396,1.287627,7.396910,1.519535,58.248300,1.358555,15.465800,0.680724
2023-01-01 15:04:00+00:00,319.673310,20.230825,0.00000,3.543093,0.088499,18533.832141,6.381349,3.106249,0.000000,528.5635,...,1.280214,1.083320,21.986393,1.287627,7.396910,1.519534,58.248206,1.358554,15.465772,0.680717


In [58]:
multidf_okx = convert_to_multi(single_okx, t_okx)
multidf_okx.head()

volume                                          \
                                  okx                                           
                             APE_USDT  AVAX_USDT AXS_USDT  BNB_USDT  BTC_USDT   
timestamp                                                                       
2023-01-01 15:00:00+00:00  143.559420  10.223912  5.74552  1.409575  0.325563   
2023-01-01 15:01:00+00:00    0.000000   0.958191  0.00000  3.170915  0.032026   
2023-01-01 15:02:00+00:00    0.000000   0.010050  0.00000  3.585290  0.279657   
2023-01-01 15:03:00+00:00   68.454178   0.010238  0.00000  7.773471  0.252629   
2023-01-01 15:04:00+00:00  319.673310  20.230825  0.00000  3.543093  0.088499   

                                                                               \
                                                                                
                               DOGE_USDT    DOT_USDT    ETH_USDT     FTM_USDT   
timestamp                                                                       
2023-01-01 15:00:00+00:00  238491.424176  355.671048  167.288462  7516.504437   
2023-01-01 15:01:00+00:00    7210.098280    0.000000    8.334945     0.000000   
2023-01-01 15:02:00+00:00    2218.514779    0.000000    2.583366     0.000000   
2023-01-01 15:03:00+00:00    1042.192843    0.000000    0.971597     0.000000   
2023-01-01 15:04:00+00:00   18533.832141    6.381349    3.106249     0.000000   

                                     ...      vwap                       \
                                     ...       okx                        
                           GMT_USDT  ...  FTM_USDT  GMT_USDT  LINK_USDT   
timestamp                            ...                                  
2023-01-01 15:00:00+00:00  249.6042  ...  1.280214  1.083322  21.986399   
2023-01-01 15:01:00+00:00    0.0000  ...  1.280214  1.083322  21.986397   
2023-01-01 15:02:00+00:00    0.0000  ...  1.280214  1.083322  21.986396   
2023-01-01 15:03:00+00:00    0.0000  ...  1.280214  1.083322  21.986396   
2023-01-01 15:04:00+00:00  528.5635  ...  1.280214  1.083320  21.986393   

                                                                     \
                                                                      
                          MATIC_USDT NEAR_USDT SAND_USDT   SOL_USDT   
timestamp                                                             
2023-01-01 15:00:00+00:00   1.287627  7.396922  1.519536  58.248381   
2023-01-01 15:01:00+00:00   1.287627  7.396922  1.519535  58.248369   
2023-01-01 15:02:00+00:00   1.287627  7.396910  1.519535  58.248303   
2023-01-01 15:03:00+00:00   1.287627  7.396910  1.519535  58.248300   
2023-01-01 15:04:00+00:00   1.287627  7.396910  1.519534  58.248206   

                                                           
                                                           
                          STORJ_USDT WAVES_USDT  XRP_USDT  
timestamp                                                  
2023-01-01 15:00:00+00:00   1.358559  15.465800  0.680726  
2023-01-01 15:01:00+00:00   1.358559  15.465800  0.680725  
2023-01-01 15:02:00+00:00   1.358555  15.465800  0.680724  
2023-01-01 15:03:00+00:00   1.358555  15.465800  0.680724  
2023-01-01 15:04:00+00:00   1.358554  15.465772  0.680717  

[5 rows x 36 columns]

# while the multiindexing is done correctly, we run into a problem... 
each coin does not exist on the exchange for the same time interval so what is the best way to create this merged df for a single exchange?
- should we have NAN values at for some coins at parts of the exchange -> could run into with merging dfs of different length
    - if we do this, we could group by currency, make our vwap calculations, and then when trying to merge grouped dfs this is where we'd run into an issue
- do we just try to find a common interval where all coins in the exchange exist?
    - we could be wasting a lot of data this way

I think the best way is option 1, so let us make some modifications 

In [7]:
def create_single(exchange_df):
    exchange_df['timestamp'] = exchange_df['timestamp'].astype('datetime64[ms]')
    exchange_df = exchange_df.sort_values(by='timestamp')
    id = exchange_df['exchange_id'].unique()[0]
    exchange_df = exchange_df.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'])
    grouped = exchange_df.groupby('currency_pair')
    dfs = [grouped.get_group(x) for x in grouped.groups]
    timestamp = pd.DataFrame(exchange_df['timestamp'].unique())
    timestamp = timestamp.rename(columns={0:"timestamp"})
    merged = timestamp.copy()
    for df in dfs:
        df[f"vwap-{id}:{df['currency_pair'].unique()[0]}"] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
        df.rename(columns={'volume' : 'volume-' + id +':'+df['currency_pair'].unique()[0]}, inplace=True)
        df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
        df.set_index('timestamp', inplace=True)
    for df in dfs:
        merged = pd.merge_asof(merged, df, on='timestamp')
    merged = merged.set_index('timestamp')
    return merged.sort_index(axis=1), timestamp

# def convert_to_multi(df, timestamp):
#     vols, vwap = "", ""
#     vol_cols, vwap_cols = list(df.loc[:, df.columns.str.startswith('volume')].columns), list(df.loc[:, df.columns.str.startswith('vwap')].columns)
#     for elem in vol_cols:
#         vols = "".join([vols, elem]) + " "
#     for elem in vwap_cols:
#         vwap = "".join([vwap, elem]) + " "
#     volume_str, vwap_str = "volume " * int(len(list(df.columns))/2), "vwap " * int(len(list(df.columns))/2)
#     new_df = pd.DataFrame(np.array(df), columns=[(volume_str + vwap_str).split(), (vols + vwap).split()])
#     # new_df.set_index(timestamp, inplace=True)
#     rename_dict = dict(zip(new_df.columns.levels[1], [word[1][7:] if word[1].startswith('volume') else word[1][5:] for word in new_df.columns]))
#     new_df = new_df.rename(columns=rename_dict, level=1)
#     new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
#     return new_df


def convert_to_multi(df, timestamp):
    # creating most inner column 
    cols_str = ""
    cols = list(df.columns)
    for elem in cols:
        cols_str = "".join([cols_str, elem]) + " "
    len(cols_str.split())

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchanges_str = ""
    for col in cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchanges_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * int(len(cols_str.split())/2)

    new_df = pd.DataFrame(np.array(df), columns=[features_str.split(), exchanges_str.split(), cols_str.split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
    return new_df

### okx

In [8]:
single_okx, t_okx = create_single(df_okx)
# single_okx.to_csv('single_okx.csv')
single_okx.head()

,volume-okx:APE_USDT,volume-okx:AVAX_USDT,volume-okx:AXS_USDT,volume-okx:BNB_USDT,volume-okx:BTC_USDT,volume-okx:DOGE_USDT,volume-okx:DOT_USDT,volume-okx:ETH_USDT,volume-okx:FTM_USDT,volume-okx:GMT_USDT,...,vwap-okx:FTM_USDT,vwap-okx:GMT_USDT,vwap-okx:LINK_USDT,vwap-okx:MATIC_USDT,vwap-okx:NEAR_USDT,vwap-okx:SAND_USDT,vwap-okx:SOL_USDT,vwap-okx:STORJ_USDT,vwap-okx:WAVES_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 15:00:00,NaN,0.0000,NaN,NaN,2.567870,1.778900e+05,19918.3194,47.074230,8088.0051,NaN,...,0.017400,NaN,12.136000,NaN,1.365400,NaN,1.642000,NaN,6.225000,0.237200
2021-01-01 15:01:00,NaN,0.0000,NaN,NaN,1.985712,1.835310e+05,13112.1579,62.386415,5591.9731,NaN,...,0.017420,NaN,12.147935,NaN,1.365355,NaN,1.641388,0.2992,6.223844,0.237389
2021-01-01 15:02:00,NaN,137.1692,NaN,NaN,35.688282,1.462586e+05,12972.3444,52.394073,8611.0840,NaN,...,0.017413,NaN,12.149790,NaN,1.365157,NaN,1.638708,0.2992,6.222253,0.237936
2021-01-01 15:03:00,NaN,33.7543,NaN,NaN,4.360614,5.917393e+05,8816.4888,38.960994,6114.4515,NaN,...,0.017410,NaN,12.153976,NaN,1.365119,NaN,1.637566,0.2992,6.218422,0.238043
2021-01-01 15:04:00,NaN,0.0000,NaN,NaN,2.620863,1.676643e+06,6131.3501,55.848012,7423.5168,NaN,...,0.017408,NaN,12.159146,NaN,1.365020,NaN,1.637444,0.2992,6.216047,0.238044


In [9]:
multidf_okx = convert_to_multi(single_okx, t_okx)
# multidf_okx.to_csv('multi_okx.csv')
multidf_okx.head()

volume                                         \
                         okx                                          
                    APE_USDT AVAX_USDT AXS_USDT BNB_USDT   BTC_USDT   
timestamp                                                             
2021-01-01 15:00:00      NaN    0.0000      NaN      NaN   2.567870   
2021-01-01 15:01:00      NaN    0.0000      NaN      NaN   1.985712   
2021-01-01 15:02:00      NaN  137.1692      NaN      NaN  35.688282   
2021-01-01 15:03:00      NaN   33.7543      NaN      NaN   4.360614   
2021-01-01 15:04:00      NaN    0.0000      NaN      NaN   2.620863   

                                                                              \
                                                                               
                        DOGE_USDT    DOT_USDT   ETH_USDT   FTM_USDT GMT_USDT   
timestamp                                                                      
2021-01-01 15:00:00  1.778900e+05  19918.3194  47.074230  8088.0051      NaN   
2021-01-01 15:01:00  1.835310e+05  13112.1579  62.386415  5591.9731      NaN   
2021-01-01 15:02:00  1.462586e+05  12972.3444  52.394073  8611.0840      NaN   
2021-01-01 15:03:00  5.917393e+05   8816.4888  38.960994  6114.4515      NaN   
2021-01-01 15:04:00  1.676643e+06   6131.3501  55.848012  7423.5168      NaN   

                     ...      vwap                                           \
                     ...       okx                                            
                     ...  FTM_USDT GMT_USDT  LINK_USDT MATIC_USDT NEAR_USDT   
timestamp            ...                                                      
2021-01-01 15:00:00  ...  0.017400      NaN  12.136000        NaN  1.365400   
2021-01-01 15:01:00  ...  0.017420      NaN  12.147935        NaN  1.365355   
2021-01-01 15:02:00  ...  0.017413      NaN  12.149790        NaN  1.365157   
2021-01-01 15:03:00  ...  0.017410      NaN  12.153976        NaN  1.365119   
2021-01-01 15:04:00  ...  0.017408      NaN  12.159146        NaN  1.365020   

                                                                         
                                                                         
                    SAND_USDT  SOL_USDT STORJ_USDT WAVES_USDT  XRP_USDT  
timestamp                                                                
2021-01-01 15:00:00       NaN  1.642000        NaN   6.225000  0.237200  
2021-01-01 15:01:00       NaN  1.641388     0.2992   6.223844  0.237389  
2021-01-01 15:02:00       NaN  1.638708     0.2992   6.222253  0.237936  
2021-01-01 15:03:00       NaN  1.637566     0.2992   6.218422  0.238043  
2021-01-01 15:04:00       NaN  1.637444     0.2992   6.216047  0.238044  

[5 rows x 36 columns]

### Binance Futures

In [10]:
single_binance_futures, t_binance_futures = create_single(df_binance_futures)
# single_binance_futures.to_csv('single_binance_futures.csv')
single_binance_futures.head()

,volume-binance_futures:APE_USDT,volume-binance_futures:AVAX_USDT,volume-binance_futures:AXS_USDT,volume-binance_futures:BAKE_USDT,volume-binance_futures:BNB_USDT,volume-binance_futures:BTC_BUSD,volume-binance_futures:BTC_USDT,volume-binance_futures:CRV_USDT,volume-binance_futures:CTK_USDT,volume-binance_futures:DOGE_USDT,...,vwap-binance_futures:MATIC_USDT,vwap-binance_futures:NEAR_USDT,vwap-binance_futures:OGN_USDT,vwap-binance_futures:RUNE_USDT,vwap-binance_futures:SAND_USDT,vwap-binance_futures:SOL_USDT,vwap-binance_futures:STORJ_USDT,vwap-binance_futures:UNFI_USDT,vwap-binance_futures:WAVES_USDT,vwap-binance_futures:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-09-08 17:57:00,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:58:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:59:00,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:01:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
multidf_binance_futures = convert_to_multi(single_binance_futures, t_binance_futures)
# multidf_binance_futures.to_csv("multi_binance_futures.csv")
multidf_binance_futures.head()

volume                                        \
                    binance_futures                                         
                           APE_USDT AVAX_USDT AXS_USDT BAKE_USDT BNB_USDT   
timestamp                                                                   
2019-09-08 17:57:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN      NaN   

                                                                   ...  \
                                                                   ...   
                    BTC_BUSD BTC_USDT CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                          ...   
2019-09-08 17:57:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 17:58:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 17:59:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 18:00:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 18:01:00      NaN    0.000      NaN      NaN       NaN  ...   

                               vwap                                         \
                    binance_futures                                          
                         MATIC_USDT NEAR_USDT OGN_USDT RUNE_USDT SAND_USDT   
timestamp                                                                    
2019-09-08 17:57:00             NaN       NaN      NaN       NaN       NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN       NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN       NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN       NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN       NaN   

                                                                       
                                                                       
                    SOL_USDT STORJ_USDT UNFI_USDT WAVES_USDT XRP_USDT  
timestamp                                                              
2019-09-08 17:57:00      NaN        NaN       NaN        NaN      NaN  
2019-09-08 17:58:00      NaN        NaN       NaN        NaN      NaN  
2019-09-08 17:59:00      NaN        NaN       NaN        NaN      NaN  
2019-09-08 18:00:00      NaN        NaN       NaN        NaN      NaN  
2019-09-08 18:01:00      NaN        NaN       NaN        NaN      NaN  

[5 rows x 54 columns]

## Binance Spot

In [12]:
single_binance_spot, t_binance_spot = create_single(df_binance_spot)
#single_binance_spot.to_csv('single_binance_spot.csv')
single_binance_spot.head() 

,volume-binance_spot:APE_USDT,volume-binance_spot:AVAX_USDT,volume-binance_spot:AXS_USDT,volume-binance_spot:BAKE_USDT,volume-binance_spot:BNB_USDT,volume-binance_spot:BTC_BUSD,volume-binance_spot:BTC_USDT,volume-binance_spot:CRV_USDT,volume-binance_spot:CTK_USDT,volume-binance_spot:DOGE_USDT,...,vwap-binance_spot:MATIC_USDT,vwap-binance_spot:NEAR_USDT,vwap-binance_spot:OGN_USDT,vwap-binance_spot:RUNE_USDT,vwap-binance_spot:SAND_USDT,vwap-binance_spot:SOL_USDT,vwap-binance_spot:STORJ_USDT,vwap-binance_spot:UNFI_USDT,vwap-binance_spot:WAVES_USDT,vwap-binance_spot:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,8358.48,3419.8,NaN,807.624,8.037298,27.457032,21622.682,65.21,76079.0,...,0.017590,1.396050,0.113250,1.168400,0.035617,1.507900,0.290650,5.347700,6.198050,0.219265
2021-01-01 00:01:00,NaN,3946.00,7397.8,NaN,2734.241,19.396598,58.477501,9453.546,249.48,764452.0,...,0.017590,1.400540,0.113257,1.169367,0.035617,1.513268,0.291044,5.347941,6.198614,0.219728
2021-01-01 00:02:00,NaN,5531.79,8846.4,NaN,778.868,5.349700,42.470329,7765.457,521.06,563854.0,...,0.017594,1.402482,0.113125,1.170026,0.035605,1.513813,0.291036,5.328001,6.199854,0.219846
2021-01-01 00:03:00,NaN,3586.61,5425.0,NaN,890.907,15.106922,30.360677,30425.292,56.27,517363.0,...,0.017594,1.402641,0.113056,1.170168,0.035605,1.513813,0.290888,5.328327,6.200300,0.219916
2021-01-01 00:04:00,NaN,3981.03,1550.6,NaN,483.711,3.272429,24.124339,2014.815,69.70,452148.0,...,0.017595,1.405933,0.113053,1.170168,0.035605,1.513811,0.290723,5.327328,6.200386,0.219989


In [13]:
multidf_binance_spot = convert_to_multi(single_binance_spot, t_binance_spot)
# multidf_binance_spot.to_csv('multi_binance_spot.csv')
multidf_binance_spot.head()

volume                                         \
                    binance_spot                                          
                        APE_USDT AVAX_USDT AXS_USDT BAKE_USDT  BNB_USDT   
timestamp                                                                 
2021-01-01 00:00:00          NaN   8358.48   3419.8       NaN   807.624   
2021-01-01 00:01:00          NaN   3946.00   7397.8       NaN  2734.241   
2021-01-01 00:02:00          NaN   5531.79   8846.4       NaN   778.868   
2021-01-01 00:03:00          NaN   3586.61   5425.0       NaN   890.907   
2021-01-01 00:04:00          NaN   3981.03   1550.6       NaN   483.711   

                                                                         ...  \
                                                                         ...   
                      BTC_BUSD   BTC_USDT   CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                                ...   
2021-01-01 00:00:00   8.037298  27.457032  21622.682    65.21   76079.0  ...   
2021-01-01 00:01:00  19.396598  58.477501   9453.546   249.48  764452.0  ...   
2021-01-01 00:02:00   5.349700  42.470329   7765.457   521.06  563854.0  ...   
2021-01-01 00:03:00  15.106922  30.360677  30425.292    56.27  517363.0  ...   
2021-01-01 00:04:00   3.272429  24.124339   2014.815    69.70  452148.0  ...   

                            vwap                                          \
                    binance_spot                                           
                      MATIC_USDT NEAR_USDT  OGN_USDT RUNE_USDT SAND_USDT   
timestamp                                                                  
2021-01-01 00:00:00     0.017590  1.396050  0.113250  1.168400  0.035617   
2021-01-01 00:01:00     0.017590  1.400540  0.113257  1.169367  0.035617   
2021-01-01 00:02:00     0.017594  1.402482  0.113125  1.170026  0.035605   
2021-01-01 00:03:00     0.017594  1.402641  0.113056  1.170168  0.035605   
2021-01-01 00:04:00     0.017595  1.405933  0.113053  1.170168  0.035605   

                                                                         
                                                                         
                     SOL_USDT STORJ_USDT UNFI_USDT WAVES_USDT  XRP_USDT  
timestamp                                                                
2021-01-01 00:00:00  1.507900   0.290650  5.347700   6.198050  0.219265  
2021-01-01 00:01:00  1.513268   0.291044  5.347941   6.198614  0.219728  
2021-01-01 00:02:00  1.513813   0.291036  5.328001   6.199854  0.219846  
2021-01-01 00:03:00  1.513813   0.290888  5.328327   6.200300  0.219916  
2021-01-01 00:04:00  1.513811   0.290723  5.327328   6.200386  0.219989  

[5 rows x 54 columns]

## Binance US

In [14]:
single_binance_us, t_binance_us = create_single(df_binance_us)
# single_binance_us.to_csv('single_binance_us.csv')
single_binance_us.head()

,volume-binanceus:APE_USDT,volume-binanceus:AVAX_USDT,volume-binanceus:AXS_USDT,volume-binanceus:BNB_USDT,volume-binanceus:BTC_BUSD,volume-binanceus:BTC_USDT,volume-binanceus:CRV_USDT,volume-binanceus:DOGE_USDT,volume-binanceus:DOT_USDT,volume-binanceus:ETH_BUSD,...,vwap-binanceus:DOT_USDT,vwap-binanceus:ETH_BUSD,vwap-binanceus:ETH_USDT,vwap-binanceus:FTM_USDT,vwap-binanceus:LINK_USDT,vwap-binanceus:NEAR_USDT,vwap-binanceus:OGN_USDT,vwap-binanceus:SAND_USDT,vwap-binanceus:SOL_USDT,vwap-binanceus:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,8.03,0.016564,0.138756,NaN,0.0,NaN,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.219180
2021-01-01 00:01:00,NaN,NaN,NaN,17.78,0.129987,1.543793,NaN,0.0,NaN,5.96981,...,NaN,738.780000,738.340000,NaN,NaN,NaN,NaN,NaN,NaN,0.220009
2021-01-01 00:02:00,NaN,NaN,NaN,10.43,0.012593,0.094843,NaN,0.0,NaN,0.00000,...,NaN,738.780000,738.354206,NaN,NaN,NaN,NaN,NaN,NaN,0.220009
2021-01-01 00:03:00,NaN,NaN,NaN,0.00,0.025558,0.042083,NaN,0.0,NaN,0.00000,...,NaN,738.780000,738.333986,NaN,NaN,NaN,NaN,NaN,NaN,0.220009
2021-01-01 00:04:00,NaN,NaN,NaN,0.00,0.008773,0.019755,NaN,0.0,NaN,1.09141,...,NaN,738.664077,738.332799,NaN,NaN,NaN,NaN,NaN,NaN,0.220009


In [15]:
multidf_binance_us = convert_to_multi(single_binance_us, t_binance_us)
# multidf_binance_us.to_csv('multi_binance_us.csv')
multidf_binance_us.head()

volume                                                  \
                    binanceus                                                   
                     APE_USDT AVAX_USDT AXS_USDT BNB_USDT  BTC_BUSD  BTC_USDT   
timestamp                                                                       
2021-01-01 00:00:00       NaN       NaN      NaN     8.03  0.016564  0.138756   
2021-01-01 00:01:00       NaN       NaN      NaN    17.78  0.129987  1.543793   
2021-01-01 00:02:00       NaN       NaN      NaN    10.43  0.012593  0.094843   
2021-01-01 00:03:00       NaN       NaN      NaN     0.00  0.025558  0.042083   
2021-01-01 00:04:00       NaN       NaN      NaN     0.00  0.008773  0.019755   

                                                          ...      vwap  \
                                                          ... binanceus   
                    CRV_USDT DOGE_USDT DOT_USDT ETH_BUSD  ...  DOT_USDT   
timestamp                                                 ...             
2021-01-01 00:00:00      NaN       0.0      NaN  0.00000  ...       NaN   
2021-01-01 00:01:00      NaN       0.0      NaN  5.96981  ...       NaN   
2021-01-01 00:02:00      NaN       0.0      NaN  0.00000  ...       NaN   
2021-01-01 00:03:00      NaN       0.0      NaN  0.00000  ...       NaN   
2021-01-01 00:04:00      NaN       0.0      NaN  1.09141  ...       NaN   

                                                                          \
                                                                           
                       ETH_BUSD    ETH_USDT FTM_USDT LINK_USDT NEAR_USDT   
timestamp                                                                  
2021-01-01 00:00:00         NaN         NaN      NaN       NaN       NaN   
2021-01-01 00:01:00  738.780000  738.340000      NaN       NaN       NaN   
2021-01-01 00:02:00  738.780000  738.354206      NaN       NaN       NaN   
2021-01-01 00:03:00  738.780000  738.333986      NaN       NaN       NaN   
2021-01-01 00:04:00  738.664077  738.332799      NaN       NaN       NaN   

                                                           
                                                           
                    OGN_USDT SAND_USDT SOL_USDT  XRP_USDT  
timestamp                                                  
2021-01-01 00:00:00      NaN       NaN      NaN  0.219180  
2021-01-01 00:01:00      NaN       NaN      NaN  0.220009  
2021-01-01 00:02:00      NaN       NaN      NaN  0.220009  
2021-01-01 00:03:00      NaN       NaN      NaN  0.220009  
2021-01-01 00:04:00      NaN       NaN      NaN  0.220009  

[5 rows x 36 columns]

# now to make the single df with all exchanges
using merge as of, i run into the problem either for ex. if i'm merging single_okx with single_binance_futures, then i can either keep all of the timestamps of one of the dataframes, but if they're nonoverlapping time intervals then this is an issue

In [16]:
merged = pd.merge(single_okx, single_binance_futures, on='timestamp', how='outer').sort_index()

In [17]:
merged = pd.merge(merged, single_binance_spot, on='timestamp', how='outer').sort_index()

In [18]:
merged = pd.merge(merged, single_binance_us, on='timestamp', how='outer').sort_index()

In [19]:
merged = merged.sort_index(axis=1)

In [22]:
merged

,volume-binance_futures:APE_USDT,volume-binance_futures:AVAX_USDT,volume-binance_futures:AXS_USDT,volume-binance_futures:BAKE_USDT,volume-binance_futures:BNB_USDT,volume-binance_futures:BTC_BUSD,volume-binance_futures:BTC_USDT,volume-binance_futures:CRV_USDT,volume-binance_futures:CTK_USDT,volume-binance_futures:DOGE_USDT,...,vwap-okx:FTM_USDT,vwap-okx:GMT_USDT,vwap-okx:LINK_USDT,vwap-okx:MATIC_USDT,vwap-okx:NEAR_USDT,vwap-okx:SAND_USDT,vwap-okx:SOL_USDT,vwap-okx:STORJ_USDT,vwap-okx:WAVES_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-09-08 17:57:00,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:58:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:59:00,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:01:00,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-31 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
multi_merged = convert_to_multi(merged, pd.DataFrame(merged.index))

In [21]:
multi_merged

volume                                        \
                    binance_futures                                         
                           APE_USDT AVAX_USDT AXS_USDT BAKE_USDT BNB_USDT   
timestamp                                                                   
2019-09-08 17:57:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN      NaN   
...                             ...       ...      ...       ...      ...   
2023-01-31 23:55:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:56:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:57:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:58:00             NaN       NaN      NaN       NaN      NaN   
2023-01-31 23:59:00             NaN       NaN      NaN       NaN      NaN   

                                                                   ...  \
                                                                   ...   
                    BTC_BUSD BTC_USDT CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                          ...   
2019-09-08 17:57:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 17:58:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 17:59:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 18:00:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 18:01:00      NaN    0.000      NaN      NaN       NaN  ...   
...                      ...      ...      ...      ...       ...  ...   
2023-01-31 23:55:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:56:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:57:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:58:00      NaN      NaN      NaN      NaN       NaN  ...   
2023-01-31 23:59:00      NaN      NaN      NaN      NaN       NaN  ...   

                        vwap                                          \
                         okx                                           
                    FTM_USDT GMT_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                              
2019-09-08 17:57:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:58:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:59:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:00:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:01:00      NaN      NaN       NaN        NaN       NaN   
...                      ...      ...       ...        ...       ...   
2023-01-31 23:55:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:56:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:57:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:58:00      NaN      NaN       NaN        NaN       NaN   
2023-01-31 23:59:00      NaN      NaN       NaN        NaN       NaN   

                                                                       
                                                                       
                    SAND_USDT SOL_USDT STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                              
2019-09-08 17:57:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:58:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:59:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:00:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:01:00       NaN      NaN        NaN        NaN      NaN  
...                       ...    

In [26]:
multi_merged.to_csv('multi_merged.csv')

In [24]:
multi_merged.columns.levels[1]

Index(['binance_futures', 'binance_spot', 'binanceus', 'okx'], dtype='object')